
# CS5228 Project - Team not found

# Data Preprocessing v2


In [45]:
# import packages
import numpy as np
import pandas as pd

In [46]:
# output numeric only
num_cols = [
    "bedrooms",
    "bathrooms",
    "lat",
    "lng",
    "since_built_year",
    "no_of_units",
    "area_size",
    "since_listing_month",
]
bin_cols = ["region", "additional_rooms", "freehold"]
cat_cols = ["name", "model", "district", "planning_area", "subszone" ]
res_col = ["price"]

assert len(num_cols+bin_cols+cat_cols) == 16


train_df = pd.read_csv('./data/train_numeric_cat.csv', index_col=0)
test_df = pd.read_csv('./data_/test_numeric_cat.csv', index_col=0)

### label encode model  




### model: label encoding

In [47]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder

In [48]:
print(train_df.model.unique())

['condominium' 'apartment' 'executive condominium' 'landed']


In [49]:

encode_order = ["executive condominium", "landed", "apartment", "condominium"]

for i, cat in enumerate(encode_order):
    train_df.loc[train_df['model']==cat, 'model'] = i
    test_df.loc[test_df['model']==cat, 'model'] = i

train_df['model'] = train_df['model'].astype('int')
test_df['model'] = test_df['model'].astype('int')

In [50]:
assert len(train_df.model.unique()) == 4
assert len(test_df.model.unique()) == 4

### remove subzone

In [51]:
train_df = train_df.drop(columns=['subszone'])
test_df = test_df.drop(columns=['subszone'])

### district, planning_area, subszone  one hot encoding 

In [52]:
from sklearn.preprocessing import OneHotEncoder

In [53]:
def my_onehot_encoder(train_XY, test_X, onehot_col_names, scaler=None):
    """
     Onehot encoding for given column names.

    """
    
    encoder = OneHotEncoder(handle_unknown='ignore')
    
    numeric_cols = [c for c in test_X.columns if c not in onehot_col_names]
    onehot_train_df, onehot_test_df = train_XY[numeric_cols].copy(), test_X[numeric_cols].copy()
    
    # keep train Y
    response_name = train_XY.columns[-1]
    train_Y = train_XY[[response_name]]
    
    
    # apply minmax or standard scaler
    if scaler:
        onehot_train_df[onehot_train_df.columns] = scaler.fit_transform(onehot_train_df)
        onehot_test_df[onehot_test_df.columns] = scaler.transform(onehot_test_df)

    
    # one-hot encoding
    for c_name in onehot_col_names:
        uniq_count = len(train_XY[c_name].unique())

        c_onehot_names = [f'{c_name}_{i}' for i in range(uniq_count)]
        
        encoder.fit(train_XY[[c_name]])
        
        
        # train
        train_output = pd.DataFrame(encoder.transform(train_XY[[c_name]]).toarray(), columns=c_onehot_names)
        onehot_train_df = pd.concat([onehot_train_df, train_output], axis=1)
        # test
        test_output = pd.DataFrame(encoder.transform(test_X[[c_name]]).toarray(), columns=c_onehot_names)
        onehot_test_df = pd.concat([onehot_test_df, test_output], axis=1)
        
    # append train Y
    onehot_train_df = pd.concat([onehot_train_df, train_Y], axis=1)

    return onehot_train_df, onehot_test_df

In [54]:
# onehot_train_df, onehot_test_df = my_onehot_encoder(train_df, test_df,  ['district','planning_area', 'subszone'])

In [55]:
# display(onehot_train_df)
# display(onehot_test_df)

In [56]:

# onehot_train_df.iloc[:,1:].to_csv('./data_clean_output/train_numeric_bin_onehot.csv', index=False)
# onehot_test_df.iloc[:,1:].to_csv('./data_clean_output/test_numeric_bin_onehot.csv', index=False)

### district, planning_area, subszone target encoding 

In [57]:
import category_encoders as ce

temp_train_df, temp_test_df = train_df.copy(), test_df.copy()

In [58]:
#Create target encoding object


# # district
encoder=ce.TargetEncoder(cols='district') 
temp_train_df[['district']] = temp_train_df[['district']].astype('object')
temp_train_df[['district']]  = encoder.fit_transform(temp_train_df[['district']],temp_train_df['price'])
temp_test_df[['district']] = encoder.transform(temp_test_df[['district']])


# # planning_area
encoder=ce.TargetEncoder(cols='planning_area') 
temp_train_df[['planning_area']]  = encoder.fit_transform(temp_train_df[['planning_area']],temp_train_df['price'])
temp_test_df[['planning_area']] = encoder.transform(temp_test_df[['planning_area']])





In [59]:
temp_train_df.iloc[:, 1:].to_csv('./data/train_numeric_bin_target.csv', index=False)
temp_test_df.iloc[:, 1:].to_csv('./data/test_numeric_bin_target.csv', index=False)

train_tar_df = temp_train_df.copy()
test_tar_df = temp_test_df.copy()

In [60]:
train_tar_df

,name,model,bedrooms,bathrooms,district,region,planning_area,lat,lng,freehold,since_built_year,no_of_units,area_size,since_listing_month,additional_rooms,price
0,4,3,3,4.0,5.058812e+06,1,5.831660e+06,1.239337,103.837487,0,11.0,151.0,2336.0,11,0,5390000.0
1,4,2,3,3.0,2.816857e+06,1,2.476742e+06,1.319533,103.847030,1,23.0,24.0,1259.0,11,0,2310000.0
2,4,3,4,3.0,2.816857e+06,1,2.476742e+06,1.315656,103.844445,1,10.0,235.0,1959.0,10,0,5279500.0
3,4,3,3,2.0,1.485024e+06,0,1.584515e+06,1.329367,103.905791,1,92.0,237.0,883.0,3,0,1843600.0
4,4,2,3,3.0,3.062476e+06,0,1.584515e+06,1.309176,103.911352,1,6.0,36.0,1066.0,4,0,2262700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,4,3,3,3.0,1.811889e+06,1,2.149466e+06,1.308896,103.863069,0,15.0,600.0,1442.0,4,0,2750000.0
26044,4,3,3,3.0,4.747986e+06,1,4.671414e+06,1.316263,103.825190,1,92.0,17.0,1561.0,3,0,6449300.0
26045,4,3,2,2.0,1.453294e+06,0,1.468193e+06,1.341303,103.953348,0,8.0,301.0,883.0,10,0,1306800.0
26046,4,3,4,3.0,1.463895e+06,0,1.400213e+06,1.335557,103.742417,0,6.0,738.0,1206.0,6,0,2420000.0


### district, planning_area, subszone label encoding

In [61]:
from sklearn.preprocessing import LabelEncoder

In [62]:
temp_train_df, temp_test_df = train_df.copy(), test_df.copy()

In [63]:
# #Create target encoding object
# encoder=LabelEncoder() 

# # district
# temp_train_df['district']  = encoder.fit_transform(temp_train_df['district'])
# temp_test_df['district'] = encoder.transform(temp_test_df['district'])


# # # planning_area
# encoder=LabelEncoder() 

# u = list(temp_train_df['planning_area'].unique())
# temp_test_df.loc[~temp_test_df['planning_area'].isin(u), 'planning_area'] = 'unknown'
# encoder.fit(u + ['unknown'])

# temp_train_df['planning_area']  = encoder.transform(temp_train_df['planning_area'])
# temp_test_df['planning_area'] = encoder.transform(temp_test_df['planning_area'])


# # # subszone
# encoder=LabelEncoder() 
# u = list(temp_train_df['subszone'].unique())
# temp_test_df.loc[~temp_test_df['subszone'].isin(u), 'subszone'] = 'unknown'
# encoder.fit(u + ['unknown'])

# temp_train_df['subszone']  = encoder.transform(temp_train_df['subszone'])
# temp_test_df['subszone'] = encoder.transform(temp_test_df['subszone'])

In [64]:
# temp_train_df.iloc[:, 1:].to_csv('./data_clean_output/train_numeric_bin_label.csv', index=False)
# temp_test_df.iloc[:, 1:].to_csv('./data_clean_output/test_numeric_bin_label.csv', index=False)

# train_label_df = temp_train_df.copy()
# test_label_df = temp_test_df.copy()

In [65]:
# display(train_label_df)
# display(test_label_df)

## Make use of auxiliary-data

### combine all auxiliary data together

In [66]:
train_distance = pd.read_csv('./data/000-auxiliary-mrt-distance-weighted.csv', index_col=0)
test_distance = pd.read_csv('./data/000-auxiliary-mrt-distance-weighted-test.csv', index_col=0)


display(train_distance)
display(test_distance)

,closeness_closest_mrt,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall,closest_mrt
0,0.026394,0.008419,5.577145,4.061902,4.042344,4.409490,3.206053,3.397070
1,0.299715,0.089016,0.408020,0.518271,0.234300,1.265343,0.339099,0.370460
2,0.208124,0.061813,0.496095,0.647547,0.294125,1.075357,0.200503,0.533492
3,0.130204,0.129623,1.845415,1.018270,0.483838,0.495765,1.680770,0.686674
4,0.064783,0.039485,2.221437,0.618157,0.374952,0.625021,0.821359,1.323576
...,...,...,...,...,...,...,...,...
26043,0.592078,0.342155,2.006256,0.348458,0.851130,1.747717,0.124455,0.172299
26044,0.265001,0.263964,2.068166,1.504327,0.611299,0.611299,1.080563,0.430559
26045,0.334392,0.091391,1.137005,1.083542,0.202554,0.098398,0.319303,0.209977
26046,0.322256,0.586456,0.894557,1.017622,0.671432,0.628858,0.272524,0.266370


,closeness_closest_mrt,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall,closest_mrt
0,0.170265,0.265983,1.659074,0.409342,0.977205,1.046077,1.129543,0.653039
1,0.104577,0.021741,2.404586,0.444060,1.462463,0.249407,0.969666,0.883631
2,0.150029,0.091443,1.714778,0.587086,0.890698,1.339162,1.467670,0.571525
3,0.051879,0.030169,1.851910,2.328437,0.707464,0.529325,2.311544,1.250516
4,0.235992,0.064245,1.174624,0.326901,1.076387,1.678925,0.394807,0.409401
...,...,...,...,...,...,...,...,...
7495,0.205507,0.078125,1.234028,0.201581,0.753366,0.590887,0.430780,0.464698
7496,0.181796,0.041782,1.769759,0.778493,1.349495,1.699094,0.921446,0.517440
7497,0.192986,0.099167,4.532308,1.691462,0.804616,0.721210,0.636956,0.376113
7498,0.383203,0.002731,1.481399,2.773995,0.631159,0.937855,0.769121,0.179380


In [67]:
train_num = pd.read_csv('./data/000-auxiliary-mrt-num-weighted.csv', index_col=0)
test_num = pd.read_csv('./data/000-auxiliary-mrt-num-weighted-test.csv', index_col=0)

display(train_num)
display(test_num)


,closeness_num_mrt_1.5km,betweenness_num_mrt_1.5km,num_comercial_3km,num_hawker_1.5km,num_primary_3km,num_secondary_3km,num_mall_2km,num_mrt_1.5km
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.565896,0.505815,3.0,4.0,17.0,10.0,12.0,5.0
2,0.456412,0.474675,3.0,5.0,16.0,7.0,35.0,4.0
3,0.447603,0.382574,1.0,2.0,17.0,9.0,2.0,5.0
4,0.178316,0.115548,1.0,4.0,12.0,10.0,8.0,2.0
...,...,...,...,...,...,...,...,...
26043,0.921843,0.538157,3.0,7.0,13.0,6.0,18.0,9.0
26044,0.348310,0.600896,1.0,0.0,13.0,11.0,15.0,3.0
26045,0.188850,0.057243,3.0,1.0,19.0,17.0,7.0,3.0
26046,0.165575,0.253552,2.0,1.0,13.0,12.0,5.0,2.0


,closeness_num_mrt_1.5km,betweenness_num_mrt_1.5km,num_comercial_3km,num_hawker_1.5km,num_primary_3km,num_secondary_3km,num_mall_2km,num_mrt_1.5km
0,0.420823,0.549154,4.0,4.0,7.0,10.0,3.0,4.0
1,0.277822,0.061204,1.0,3.0,5.0,6.0,3.0,3.0
2,0.178316,0.115548,1.0,3.0,15.0,12.0,12.0,2.0
3,0.064876,0.037727,4.0,0.0,13.0,9.0,0.0,1.0
4,0.871747,0.374799,2.0,7.0,5.0,2.0,18.0,9.0
...,...,...,...,...,...,...,...,...
7495,1.113541,0.571727,2.0,11.0,11.0,5.0,41.0,11.0
7496,0.590142,0.248702,2.0,2.0,7.0,5.0,20.0,6.0
7497,0.736355,0.619808,0.0,0.0,17.0,14.0,9.0,11.0
7498,0.284551,0.026294,3.0,0.0,14.0,6.0,3.0,4.0


In [68]:
train_aux = pd.concat([train_distance, train_num], axis=1)
test_aux =  pd.concat([test_distance, test_num], axis=1)

In [69]:
train_aux.shape

(26048, 16)

### combine one-hot data and auxiliary data together

In [70]:
# train_aux_oh = pd.concat([train_aux, onehot_train_df], axis=1).dropna()
# test_aux_oh = pd.concat([test_aux, onehot_test_df], axis=1).dropna()


# train_aux_oh.to_csv('./data_clean_output/train_aux_oh.csv', index=False)
# test_aux_oh.to_csv('./data_clean_output/test_aux_oh.csv',  index=False)

In [71]:
# display(train_aux_oh)
# display(test_aux_oh)

# # train_aux_oh.isnull().any().any()
# print(train_aux)
# print(onehot_train_df)

### combine target encode data and auxiliary data together

In [72]:
train_aux_tar = pd.merge(train_aux, train_tar_df, left_index=True, right_index=True)
test_aux_tar = pd.merge(test_aux, test_tar_df, left_index=True, right_index=True)


train_aux_tar.to_csv('./data/train_aux_tar.csv', index=False)
test_aux_tar.to_csv('./data/test_aux_tar.csv', index=False)

In [73]:
display(train_aux_tar)
display(test_aux_tar)

,closeness_closest_mrt,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall,closest_mrt,closeness_num_mrt_1.5km,betweenness_num_mrt_1.5km,...,planning_area,lat,lng,freehold,since_built_year,no_of_units,area_size,since_listing_month,additional_rooms,price
0,0.026394,0.008419,5.577145,4.061902,4.042344,4.409490,3.206053,3.397070,0.000000,0.000000,...,5.831660e+06,1.239337,103.837487,0,11.0,151.0,2336.0,11,0,5390000.0
1,0.299715,0.089016,0.408020,0.518271,0.234300,1.265343,0.339099,0.370460,0.565896,0.505815,...,2.476742e+06,1.319533,103.847030,1,23.0,24.0,1259.0,11,0,2310000.0
2,0.208124,0.061813,0.496095,0.647547,0.294125,1.075357,0.200503,0.533492,0.456412,0.474675,...,2.476742e+06,1.315656,103.844445,1,10.0,235.0,1959.0,10,0,5279500.0
3,0.130204,0.129623,1.845415,1.018270,0.483838,0.495765,1.680770,0.686674,0.447603,0.382574,...,1.584515e+06,1.329367,103.905791,1,92.0,237.0,883.0,3,0,1843600.0
4,0.064783,0.039485,2.221437,0.618157,0.374952,0.625021,0.821359,1.323576,0.178316,0.115548,...,1.584515e+06,1.309176,103.911352,1,6.0,36.0,1066.0,4,0,2262700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,0.592078,0.342155,2.006256,0.348458,0.851130,1.747717,0.124455,0.172299,0.921843,0.538157,...,2.149466e+06,1.308896,103.863069,0,15.0,600.0,1442.0,4,0,2750000.0
26044,0.265001,0.263964,2.068166,1.504327,0.611299,0.611299,1.080563,0.430559,0.348310,0.600896,...,4.671414e+06,1.316263,103.825190,1,92.0,17.0,1561.0,3,0,6449300.0
26045,0.334392,0.091391,1.137005,1.083542,0.202554,0.098398,0.319303,0.209977,0.188850,0.057243,...,1.468193e+06,1.341303,103.953348,0,8.0,301.0,883.0,10,0,1306800.0
26046,0.322256,0.586456,0.894557,1.017622,0.671432,0.628858,0.272524,0.266370,0.165575,0.253552,...,1.400213e+06,1.335557,103.742417,0,6.0,738.0,1206.0,6,0,2420000.0


,closeness_closest_mrt,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall,closest_mrt,closeness_num_mrt_1.5km,betweenness_num_mrt_1.5km,...,region,planning_area,lat,lng,freehold,since_built_year,no_of_units,area_size,since_listing_month,additional_rooms
0,0.170265,0.265983,1.659074,0.409342,0.977205,1.046077,1.129543,0.653039,0.420823,0.549154,...,1,2.509983e+06,1.313566,103.803218,1,62.0,638.0,710.0,4,0
1,0.104577,0.021741,2.404586,0.444060,1.462463,0.249407,0.969666,0.883631,0.277822,0.061204,...,1,2.149466e+06,1.298437,103.884408,1,6.0,130.0,1055.0,6,0
2,0.150029,0.091443,1.714778,0.587086,0.890698,1.339162,1.467670,0.571525,0.178316,0.115548,...,0,1.584515e+06,1.317851,103.908905,1,9.0,157.0,463.0,4,0
3,0.051879,0.030169,1.851910,2.328437,0.707464,0.529325,2.311544,1.250516,0.064876,0.037727,...,0,1.400474e+06,1.358005,103.965725,0,92.0,428.0,645.0,4,0
4,0.235992,0.064245,1.174624,0.326901,1.076387,1.678925,0.394807,0.409401,0.871747,0.374799,...,1,4.383065e+06,1.277474,103.849419,0,5.0,510.0,6200.0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.205507,0.078125,1.234028,0.201581,0.753366,0.590887,0.430780,0.464698,1.113541,0.571727,...,1,3.501386e+06,1.289610,103.841276,0,23.0,492.0,743.0,4,0
7496,0.181796,0.041782,1.769759,0.778493,1.349495,1.699094,0.921446,0.517440,0.590142,0.248702,...,1,2.149466e+06,1.296610,103.867069,0,25.0,737.0,1345.0,5,0
7497,0.192986,0.099167,4.532308,1.691462,0.804616,0.721210,0.636956,0.376113,0.736355,0.619808,...,0,1.329559e+06,1.375255,103.757531,0,92.0,74.0,1033.0,3,1
7498,0.383203,0.002731,1.481399,2.773995,0.631159,0.937855,0.769121,0.179380,0.284551,0.026294,...,0,1.437086e+06,1.398588,103.874632,0,1.0,735.0,581.0,4,1


In [74]:
assert (train_aux_tar.index == train_df.index).all()

## normalisation & standardisation 

https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


- Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.
- Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Also, unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.

In [75]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from typing import AnyStr, Callable

In [76]:
def my_scaler(train_XY:pd.DataFrame, test_X:pd.DataFrame, scale_col_names:list, scaler:Callable):
    
    
    norm_train_df, norm_test_df = train_XY.copy(), test_X.copy()
    
    train_scale_X = train_XY[scale_col_names]
    test_scale_X = test_X[scale_col_names]
    
    normalizer = scaler.fit(train_scale_X)
    norm_train_X = normalizer.transform(train_scale_X)
    norm_test_X = normalizer.transform(test_scale_X)
    
    norm_train_df[scale_col_names] = norm_train_X
    norm_test_df[scale_col_names] = norm_test_X
    
    return norm_train_df, norm_test_df



### For numeric only

In [77]:
train_num_df = pd.read_csv('./data/train_numeric.csv')
test_num_df = pd.read_csv('./data/test_numeric.csv')

In [78]:
# norm
train_num_df_norm, test_num_df_norm = my_scaler(train_num_df,test_num_df,test_num_df.columns,MinMaxScaler())
train_num_df_norm.to_csv('./data/train_numeric_norm.csv', index=False)
test_num_df_norm.to_csv('./data/test_numeric_norm.csv', index=False)

In [79]:
# stand
train_num_df_stand, test_num_df_stand = my_scaler(train_num_df,test_num_df,test_num_df.columns,StandardScaler())
train_num_df_stand.to_csv('./data/train_numeric_stand.csv', index=False)
test_num_df_stand.to_csv('./data/test_numeric_stand.csv', index=False)

### for aux + target encoding

In [80]:
# for aux + target encoding
to_scale = [
    # numeric
    "bedrooms",
    "bathrooms",
    "lat",
    "lng",
    "since_built_year",
    "no_of_units",
    "area_size",
    "since_listing_month",
    # target
    "district", 
    "planning_area", 
] + list(train_aux_tar.columns)[:16]

print(to_scale)


# normalize
train_aux_tar_norm, test_aux_tar_norm = my_scaler(train_aux_tar, test_aux_tar, to_scale, MinMaxScaler())

train_aux_tar_norm.to_csv('./data/train_aux_tar_norm.csv', index=False)
test_aux_tar_norm.to_csv('./data/test_aux_tar_norm.csv', index=False)


# train_aux_tar_norm.describe()

['bedrooms', 'bathrooms', 'lat', 'lng', 'since_built_year', 'no_of_units', 'area_size', 'since_listing_month', 'district', 'planning_area', 'closeness_closest_mrt', 'betweenness_closest_mrt', 'closest_comercial', 'closest_hawker', 'closest_primary', 'closest_secondary', 'closest_mall', 'closest_mrt', 'closeness_num_mrt_1.5km', 'betweenness_num_mrt_1.5km', 'num_comercial_3km', 'num_hawker_1.5km', 'num_primary_3km', 'num_secondary_3km', 'num_mall_2km', 'num_mrt_1.5km']


In [81]:
# standardise
train_aux_tar_stand, test_aux_tar_stand = my_scaler(train_aux_tar, test_aux_tar,  to_scale, StandardScaler())

train_aux_tar_stand.to_csv('./data/train_aux_tar_stand.csv', index=False)
test_aux_tar_stand.to_csv('./data/test_aux_tar_stand.csv', index=False)


### for aux + onehot encoding

In [82]:

# to_scale = [
#     # numeric
#     "bedrooms",
#     "bathrooms",
#     "lat",
#     "lng",
#     "since_built_year",
#     "no_of_units",
#     "area_size",
#     "since_listing_month",
# ] + list(train_aux_oh.columns)[:16]



In [83]:
# ############# normalize


# # append onehot back
# train_aux_oh_norm, test_aux_oh_norm = my_scaler(train_aux_oh, test_aux_oh, to_scale, MinMaxScaler())


# train_aux_oh_norm.to_csv('./data_clean_output/train_aux_oh_norm.csv', index=False)
# test_aux_oh_norm.to_csv('./data_clean_output/test_aux_oh_norm.csv', index=False)


In [84]:
############## standardise

# train_aux_oh_stand, test_aux_oh_stand = my_scaler(train_aux_oh, test_aux_oh, to_scale, StandardScaler())

# train_aux_oh_stand.to_csv('./data_clean_output/train_aux_oh_stand.csv', index=False)
# test_aux_oh_stand.to_csv('./data_clean_output/test_aux_oh_stand.csv', index=False)



In [85]:
# display(train_aux_oh_stand)
# display(test_aux_oh_stand)

## PCA for feature reduction

In [86]:
# from sklearn.decomposition import PCA

In [87]:
# def my_pca(train_XY, test_X, pca_func, scaler=None):
    
#     # keep train Y
#     response_name = train_XY.columns[-1]
#     train_Y = train_XY[[response_name]]
    
#     # get train X
#     train_X = train_XY.iloc[:, :-1] 
    
#     # fit training X
#     pca_func.fit(train_X)
#     # transform train X and test X
#     train_pca_X = pca_func.transform(train_X)
#     test_pca_X = pca_func.transform(test_X)
    
#     # to dataframe
#     d = train_pca_X.shape[1]
#     col_names = [f'PC_{i}' for i in range(d)]
    
#     train_pca_X = pd.DataFrame(train_pca_X, columns=col_names)
#     test_pca_X = pd.DataFrame(test_pca_X, columns=col_names)
    
#     # append train Y
#     train_pca_XY = pd.concat([train_pca_X, train_Y], axis=1)
    
    
#     return train_pca_XY, test_pca_X

In [88]:
# norm + pca
# train_aux_tar_norm_pca, test_aux_tar_norm_pca = my_pca(train_aux_tar_norm, test_aux_tar_norm, PCA(0.95))

# train_aux_tar_norm_pca.to_csv('./data_clean_output/train_aux_tar_norm_pca.csv', index=False)
# test_aux_tar_norm_pca.to_csv('./data_clean_output/test_aux_tar_norm_pca.csv', index=False)


# # stand + pca
# train_aux_tar_stand_pca, test_aux_tar_stand_pca = my_pca(train_aux_tar_stand, test_aux_tar_stand, PCA(0.95))
# train_aux_tar_stand_pca.to_csv('./data_clean_output/train_aux_tar_stand_pca.csv',index=False)
# test_aux_tar_stand_pca.to_csv('./data_clean_output/test_aux_tar_stand_pca.csv',index=False)